### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [2]:
!pip install implicit lightfm

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 307kB 15.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419514 sha256=5d83c6e33f29d4cc608f4b07b4caad874948fdbbdd3c9e3566f1147e3d17e40c
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707622 sha256=2460c93c5f435c6f084e7fdcd39607c6d95328287acece4ba6b7b19227e3d383
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built implicit lightfm


In [3]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
datapath = "/content/drive/My Drive/AU/RecSys/ml-1m/"

In [6]:
ratings = pd.read_csv(datapath + 'ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [7]:
movie_info = pd.read_csv(datapath + 'movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [8]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [11]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [14]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [15]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [18]:
get_similars(1, model)

['0    Toy Story (1995)',
 '369    Red Rock West (1992)',
 '2284    Enemy of the State (1998)',
 'Series([], )',
 '1160    Double Life of Veronique, The (La Double Vie d...',
 '2275    Runaway Train (1985)',
 '1943    Back to the Future Part III (1990)',
 '1299    Kids of Survival (1993)',
 '2429    My Favorite Martian (1999)',
 '627    Land and Freedom (Tierra y libertad) (1995)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [19]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [20]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [21]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [22]:
get_recommendations(4, model)

['740    Dr. Strangelove or: How I Learned to Stop Worr...',
 '3859    Bank Dick, The (1940)',
 '1345    Crucible, The (1996)',
 '1129    Snowriders (1996)',
 '1190    Apocalypse Now (1979)',
 '1299    Kids of Survival (1993)',
 '2692    Iron Giant, The (1999)',
 '3290    Breaking Away (1979)',
 '2061    Atlantic City (1980)',
 '3776    Easy Money (1983)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [12]:
from tqdm.autonotebook import trange, tqdm

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [13]:
ex_user_item = sp.coo_matrix((ratings.rating, (ratings.user_id, ratings.movie_id)))
ex_user_item_t_csr = user_item.T.tocsr()
ex_user_item_csr = user_item.tocsr()

In [99]:
class SVD_SGD:
    def __init__(self, dim=64, iters=1e6, eps=1e-2, lmbda=1e-2, theta=1e-1):
        self.dim = dim
        self.iters = int(iters)
        self.eps = eps
        self.lmbda = lmbda
        self.theta = theta
        self.U = None
        self.V = None
        self.mu = None
        self.bu = None
        self.bv = None
    
    def fit(self, user_item):
        n_users, n_items = user_item.shape
        self.U = np.random.uniform(0, 1/np.sqrt(self.dim), (n_users, self.dim))
        self.V = np.random.uniform(0, 1/np.sqrt(self.dim), (n_items, self.dim))
        self.mu = user_item.mean()
        self.bu = np.array(user_item.mean(axis=1)).reshape(-1)
        self.bv = np.array(user_item.mean(axis=0)).reshape(-1)
        t = trange(self.iters)
        for iter in t:
            i, j = np.random.randint(0, n_users), np.random.randint(0, n_items)
            error = self.score(i, j) - user_item[i, j]
            self.U[i] -= self.eps * (error * self.V[j] + self.lmbda * self.U[i])
            self.V[j] -= self.eps * (error * self.U[i] + self.lmbda * self.V[j])
            self.mu -= self.eps * error
            self.bu[i] -= self.eps * (error + self.theta * self.bu[i])
            self.bv[j] -= self.eps * (error + self.theta * self.bv[j])

            if (iter + 1) % 100000 == 0:
                self.rmse(user_item, t)

    def rmse(self, user_item, t):
        loss = []
        i_nonzero, j_nonzero = user_item.nonzero()
        idxs = np.random.randint(len(i_nonzero), size=10000)
        for i, j in map(lambda x: (i_nonzero[x], j_nonzero[x]), idxs):
            error = self.score(i, j) - user_item[i, j]
            loss.append(error ** 2)
        t.set_postfix({'loss': np.sqrt(np.mean(loss))})
            
    def recommend(self, user_id, user_item, top_n=10):
        recommended_items = set(user_item[user_id].nonzero()[1])
        return np.argsort([self.score(user_id, j) 
            for j in range(self.V.shape[0]) if j not in recommended_items])[:-top_n:-1]

    def similar_items(self, item_id, top_n=10):
        norm = np.linalg.norm(self.V[item_id]) + 1e-5
        score = self.V @ self.V[item_id] / (np.linalg.norm(self.V, axis=1) + 1e-5) / norm
        return sorted([i for i in range(self.V.shape[0])], 
                      key=lambda i: -score[i])[:top_n]

    def score(self, i, j):
        return self.U[i] @ self.V[j] + self.bu[i] + self.bv[j] + self.mu

In [100]:
SVD_model = SVD_SGD(iters=1e7)
SVD_model.fit(user_item_csr)

In [101]:
movie_info.loc[SVD_model.similar_items(40)]

,movie_id,name,category
40,41,Richard III (1995),Drama|War
2488,2557,I Stand Alone (Seul contre tous) (1998),Drama
3464,3533,"Actor's Revenge, An (Yukinojo Henge) (1963)",Drama
2937,3006,"Insider, The (1999)",Drama
258,261,Little Women (1994),Drama
1208,1226,"Quiet Man, The (1952)",Comedy|Romance
588,592,Batman (1989),Action|Adventure|Crime|Drama
2391,2460,"Texas Chainsaw Massacre 2, The (1986)",Horror
1035,1049,"Ghost and the Darkness, The (1996)",Action|Adventure
84,85,Angels and Insects (1995),Drama|Romance


In [102]:
movie_info.loc[SVD_model.recommend(4, user_item_csr)]

,movie_id,name,category
2845,2914,Molly (1999),Comedy|Drama
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
591,595,Beauty and the Beast (1991),Animation|Children's|Musical
2749,2818,Iron Eagle IV (1995),Action|War
587,591,Tough and Deadly (1995),Action|Drama|Thriller
50,51,Guardian Angel (1994),Action|Drama|Thriller
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
606,610,Heavy Metal (1981),Action|Adventure|Animation|Horror|Sci-Fi
525,529,Searching for Bobby Fischer (1993),Drama


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [103]:
class ALS(SVD_SGD):
    def __init__(self, iters=3, eps=30, **kargs):
        super().__init__(iters=iters, eps=eps, **kargs)

    def fit(self, user_item):
        user_item_t = user_item.transpose()
        n_users, n_items = user_item.shape
        self.U = np.random.uniform(0, 1/np.sqrt(self.dim), (n_users, self.dim))
        self.V = np.random.uniform(0, 1/np.sqrt(self.dim), (n_items, self.dim))
        t = trange(self.iters)
        for iter in t:
            self.rmse(user_item, t)
            self.train(user_item, self.U, self.V)
            self.train(user_item.T, self.V, self.U)

    def train(self, user_item, U, V):
        VtV = V.T @ V
        for i in trange(U.shape[0]):
            r_u = user_item.getrow(i).toarray()[0]
            C_u = sp.diags(1 + self.eps * r_u, offsets=0)           
            p_u = r_u > 0
            VtC_uV = (VtV + V.T.dot(sp.csr_matrix.dot(C_u - sp.identity(V.shape[0]), V)))
            inv = np.linalg.inv(VtC_uV + self.lmbda * sp.identity(VtC_uV.shape[0]))
            U[i, :] = inv @ V.T @ C_u @ p_u
    
    def score(self, i, j):
        return self.U[i] @ self.V[j]

In [104]:
model = ALS()
model.fit(user_item_csr)

In [92]:
movie_info.loc[model.similar_items(0)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in true_divide


,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
3,4,Waiting to Exhale (1995),Comedy|Drama
528,532,Serial Mom (1994),Comedy|Crime|Horror
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
252,255,"Jerky Boys, The (1994)",Comedy
586,590,Dances with Wolves (1990),Adventure|Drama|Western
543,547,Surviving the Game (1994),Action|Adventure|Thriller
46,47,Seven (Se7en) (1995),Crime|Thriller
256,259,Kiss of Death (1995),Crime|Drama|Thriller
518,522,Romper Stomper (1992),Action|Drama


In [83]:
movie_info.loc[model.recommend(4, user_item_csr)]

,movie_id,name,category
856,867,Carpool (1996),Comedy|Crime
1214,1233,"Boat, The (Das Boot) (1981)",Action|Drama|War
587,591,Tough and Deadly (1995),Action|Drama|Thriller
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1195,1213,GoodFellas (1990),Crime|Drama
1204,1222,Full Metal Jacket (1987),Action|Drama|War
539,543,So I Married an Axe Murderer (1993),Comedy|Romance|Thriller
1944,2013,"Poseidon Adventure, The (1972)",Action|Adventure
110,112,Rumble in the Bronx (1995),Action|Adventure|Crime


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных